For installation of **DSSEX** please follow [installation instructions](./readme.ipynb#Install-DSSEX) or run the next cell.

In [1]:
# in case you want to install dssex in the environment of Jupyter e.g. Google Colab
%pip install -q dssex

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import dssex
from egrid import create_objects, make_data_frames, check_frames, model_from_frames
import pandas as pd
pd.set_option('display.max_colwidth', None)

def print_frames(frames):
    """Prints pandas.DataFrame instances stored in a dict"""
    messages = frames.get('Message')
    if messages is not None:
        is_error = 1 < messages.level
        errors = messages.loc[is_error].message
        if not errors.empty:
            print('Errors\n')
            for _, error in errors.items():
                print()
                print(error)
            print()
    for key,df in frames.items():
        print()
        print(key)
        display(df)
        
def print_results(res):
    """Prints values returned from function dssex.estimate"""
    messages, vals = res
    if not messages.empty:
        print('\n\n')
        print('Messages')
        display(messages)
    last_index = len(vals) - 2
    for idx, df in enumerate(vals, -1):
        print('\n\n')
        print(f'{"RESULT >> " if idx==last_index else ""}Optimization Step: {idx}')
        for name in ['nodes', 'branches', 'injections']:
            print()
            print(name.title())
            display(df[name])    

# Volt-Var-Control

In [3]:
schema00 = """
slack +-----------transformer-----------+ n +-----------> heating
      Tlink=tap    y_lo=1e3-1e3j          |               P10=200
                   y_tr=1e-6+1e-6j        |
                                          | 
                                          | 
                         cap ||---------+ n +-------((~)) motor
                          Q10=-10                          P10=160
                          Exp_v_q=2                        Q10=10

#.
Deft(id=tap type=var min=-16 max=16 value=0 is_discrete=True cost=.03)
Defk(id=kcap type=var min=0 max=5 value=0 is_discrete=True cost=.05)
Klink(id_of_injection=cap id_of_factor=kcap part=q)
Vlimit(min=.95)
"""
frames00 = make_data_frames(create_objects(schema00))
print_frames(frames00)


Branch


C:\Users\live\venv\3_10\lib\site-packages\egrid\builder.py:495: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframes[Factor.__name__] = pd.concat(
C:\Users\live\venv\3_10\lib\site-packages\egrid\builder.py:510: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframes[Vlimit.__name__] = pd.concat([vlimits, vlimits2])


,id,id_of_node_A,id_of_node_B,y_lo,y_tr
0,transformer,slack,n,1000.0-1000.0j,0.000001+0.000001j



Slacknode


,id_of_node,V
0,slack,1.0+0.0j


...

,message,level



Injectionlink


,injid,part,id,step
0,cap,q,kcap,-1



Terminallink


,branchid,nodeid,id,step
0,transformer,slack,tap,-1


In [4]:
pd.DataFrame.from_records(check_frames(frames00), columns=['message','level'])

,message,level
0,count of slack-nodes: 1,0
1,count of branches: 1,0
2,count of injections: 3,0


[step_params](./help_params.ipynb)

In [5]:
model00 = model_from_frames(frames00)
step_params=[
    # first step: minimize voltage violations
    dict(objectives='U', constraints='U'),
    # second step: minimize cost
    dict(objectives='CL', constraints='U', floss=2)
]

results00 = dssex.estimate(model00, step_params)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       10
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        4
Total number of inequality c

CasADi - 2024-01-03 20:30:37 WARNING("solver:nlp_grad failed: NaN detected for output grad_gamma_p, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-01-03 20:30:37 WARNING("Failed to calculate multipliers") [.../casadi/core/nlpsol.cpp:798]
CasADi - 2024-01-03 20:30:37 WARNING("solver:nlp_grad failed: NaN detected for output grad_gamma_p, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-01-03 20:30:37 WARNING("Failed to calculate multipliers") [.../casadi/core/nlpsol.cpp:798]
C:\Users\live\venv\3_10\lib\site-packages\dssex\result.py:827: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  branches = pd.concat([branch_res, bridgeterminal_res])
C:\Users\live\venv\3_10\lib\site-packages\dssex\re

Results for two steps. 

- Initial power flow calculation
- Optimization

In [ ]:
print_results(results00)